<a href="https://colab.research.google.com/github/vkp1533/FeatureMap/blob/main/ML_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Implementing Single Perceptron over wine dataset
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
# Load wine dataset
train = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_train.csv')
test  = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_test.csv')
# X = features, y = target
X_tr = train.drop('target', axis=1).values  # (5000, 13)
X_te = test.drop('target', axis=1).values   # (1500, 13)
y_tr = train['target'].values.reshape(-1, 1)  # (5000, 1)
y_te = test['target'].values.reshape(-1, 1)    # (1500, 1)
# Z-score scaling
mean = X_tr.mean(axis=0)
std  = X_tr.std(axis=0)
std[std == 0] = 1
X_tr = (X_tr - mean) / std
X_te = (X_te - mean) / std

# Perceptron weights (1 neuron)
np.random.seed(42)
n_feat = X_tr.shape[1]
# Input -> Output weights
w = np.random.randn(n_feat, 1) * 0.1
b = 0.0

for epoch in range(300):
    for i in range(len(X_tr)):
        # Forward: z = x·w + b
        z = X_tr[i] @ w + b
        pred = 1 if z >= 0 else 0           # Step function (0 or 1)

        # Error
        err = y_tr[i] - pred

        # Update: w = w + lr * err * x
        w += 0.1 * err * X_tr[i].reshape(-1, 1)
        b += 0.1 * err
# Test prediction
z_te = X_te @ w + b
pred = (z_te >= 0).astype(int)          # 1 if z >= 0 else 0

accuracy = np.mean(pred == y_te) * 100
print(f"\nPERCEPTRON ACCURACY (Wine): {accuracy:.2f}%")

In [31]:
#implementing multilayer perceptron
import numpy as np
import pandas as pd

# Loading wine training and testing datasets
train = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_train.csv')
test  = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_test.csv')

# X for features, y for target (0/1)
# Data preprocessing
X_tr = train.drop('target', axis=1).values
X_te = test.drop('target', axis=1).values
y_tr = train['target'].values.reshape(-1, 1)
y_te = test['target'].values.reshape(-1, 1)

# Z-score scaling (mean=0, std=1)
mean = X_tr.mean(axis=0)  # Mean of features
# Standard deviation of each feature
std  = X_tr.std(axis=0)
# Prevent divide by zero
std[std == 0] = 1


X_tr = (X_tr - mean) / std
# Scaled test features (same mean/std)
X_te = (X_te - mean) / std

# MLP weights
np.random.seed(42)
 # total no. of features
n_feat = X_tr.shape[1]

W1 = np.random.randn(n_feat, 16) * 0.5   # Input -> Hidden (13 x 16)
b1 = np.zeros((1, 16))                   # Hidden bias
W2 = np.random.randn(16, 1) * 0.5        # Hidden -> Output (16 x 1)
b2 = np.zeros((1, 1))                    # Output bias
for epoch in range(150):
    # Forward pass
    Z1 = X_tr @ W1 + b1
    # Linear (13 -> 16)
    A1 = np.maximum(0, Z1)
    # ReLU activation
    Z2 = A1 @ W2 + b2
    # Linear (16 -> 1)
    A2 = 1 / (1 + np.exp(-Z2))             # Sigmoid output (0 to 1)

    # Gradients calc
    # Output error
    err = A2 - y_tr
    # Gradient for W2
    dW2 = A1.T @ err / len(X_tr)
    # Gradient for W1 (ReLU deriv = Z1>0)
    dW1 = X_tr.T @ (err @ W2.T * (Z1 > 0)) / len(X_tr)
    # Update weights (learning rate = 0.1)
    W1 -= 0.1 * dW1
    W2 -= 0.1 * dW2

# Testing
Z1_te = X_te @ W1 + b1
A1_te = np.maximum(0, Z1_te)
A2_te = 1 / (1 + np.exp(-(A1_te @ W2 + b2)))
# keeping threshold 0.5
pred = (A2_te > 0.5).astype(int)

accuracy = np.mean(pred == y_te) * 100
print(f"\nMLP ACCURACY (Wine): {accuracy:.2f}%")


MLP ACCURACY (Wine): 100.00%


In [13]:
import pandas as pd
import numpy as np

# 1. DATA LOAD
train_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_test.csv')

X_train = train_df.iloc[:, 1:].values
y_train = np.where(train_df['target'].values == 0, -1, 1)
X_test = test_df.iloc[:, 1:].values
y_test = np.where(test_df['target'].values == 0, -1, 1)

# 2. NORMALIZE (important for soft margin)
X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_test = (X_test - X_test.mean(axis=0)) / X_test.std(axis=0)

# 3. SOFT MARGIN SVM – SIRF 10 LINES!
def soft_margin_svm(X, y, C=1.0, learning_rate=0.001, epochs=1000):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)   # w shuru mein 0
    b = 0                      # b = 0

    for _ in range(epochs):
        for i in range(n_samples):
            # Condition: y_i * (w·x_i + b) >= 1
            condition = y[i] * (np.dot(X[i], w) + b) >= 1

            if condition:
                # Sirf w ko update karo (galti nahi)
                w -= learning_rate * (2 * 0.01 * w)  # L2 regularization
            else:
                # Galti hai → w aur b dono update
                w -= learning_rate * (2 * 0.01 * w - y[i] * X[i])
                b -= learning_rate * (-y[i])

    return w, b

# 4. TRAIN KARO (C = 1.0 → thoda soft)
w, b = soft_margin_svm(X_train, y_train, C=1.0, epochs=500)

# 5. PREDICT
def predict(X, w, b):
    return np.sign(X @ w + b)

y_pred = predict(X_test, w, b)
accuracy = np.mean(y_pred == y_test)

print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 100.00%


In [12]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_test.csv')
# 2. X aur y
X_train = train.iloc[:, 1:].values
y_train = train['target'].values
X_test = test.iloc[:, 1:].values
y_test = test['target'].values

# 3. Normalize
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

# 4. K select (odd)
n_test = len(X_test)
k = int(np.sqrt(n_test))
if k % 2 == 0:
    k += 1
print(f"K = {k}")

# 5. Distance function
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

# 6. KNN predict
def knn_predict(test_point, X_train, y_train, k):
    distances = []
    for i in range(len(X_train)):
        dist = euclidean_distance(test_point, X_train[i])
        distances.append((dist, y_train[i]))
    distances.sort(key=lambda x: x[0])
    neighbors = distances[:k]
    count_0 = sum(1 for _, label in neighbors if label == 0)
    count_1 = k - count_0
    return 0 if count_0 > count_1 else 1

# 7. Predict all
y_pred = [knn_predict(X_test[i], X_train, y_train, k) for i in range(len(X_test))]
y_pred = np.array(y_pred)

# 8. Accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

K = 7
Accuracy: 97.22%


In [36]:
# PCA
#Implementing
import pandas as pd
import numpy as np

# 1. Load data
train_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/adult_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/adult_test.csv')

# 2. Features only
X_train = train.iloc[:, 1:].values
X_test = test.iloc[:, 1:].values

# 3. Mean centering
mean_vector = X_train.mean(axis=0)
X_centered = X_train - mean_vector

# 4. Covariance matrix
cov_matrix = np.cov(X_centered, rowvar=False)

# 5. Eigen decomposition
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# 6. Sort by importance
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

# 7. Explained variance
total = eigenvalues.sum()
exp_var = eigenvalues / total
cum_var = np.cumsum(exp_var)

print("Explained Variance:")
for i in range(5):
    print(f"PC{i+1}: {exp_var[i]*100:.2f}%")
print(f"First 2 PCs: {cum_var[1]*100:.2f}%")

# 8. Choose top 2
n_components = 2
W = eigenvectors[:, :n_components]

# 9. Project data
X_pca_train = X_centered @ W
X_pca_test = (X_test - mean_vector) @ W

print(f"\nOriginal: {X_train.shape} → PCA: {X_pca_train.shape}")

Explained Variance:
PC1: 99.76%
PC2: 0.22%
PC3: 0.01%
PC4: 0.01%
PC5: 0.00%
First 2 PCs: 99.98%

Original: (36, 13) → PCA: (36, 2)


In [37]:
# SVD
import pandas as pd
import numpy as np

# 1. Load data
train_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/adult_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/ML_Datasets/adult_test.csv')

# 2. Numeric columns only
cols = [1, 2, 4, 10, 11, 12]  # age, fnlwgt, edu-num, cap-gain, cap-loss, hours
X_train = train.iloc[:, cols].values

# 3. Mean centering
mean_vec = X_train.mean(axis=0)
X_centered = X_train - mean_vec

# 4. SVD from scratch
def svd_scratch(X):
    A = X @ X.T
    eig_U, U = np.linalg.eig(A)
    B = X.T @ X
    eig_V, V = np.linalg.eig(B)
    idx_U = eig_U.argsort()[::-1]
    idx_V = eig_V.argsort()[::-1]
    U = U[:, idx_U]
    V = V[:, idx_V]
    S_vals = np.sqrt(np.abs(eig_V[idx_V]))
    S = np.zeros((U.shape[1], V.shape[0]))
    min_dim = min(len(S_vals), S.shape[0], S.shape[1])
    S[:min_dim, :min_dim] = np.diag(S_vals[:min_dim])
    return U, S, V.T

# 5. Run SVD
U, S, Vt = svd_scratch(X_centered)

# 6. Results
print(f"X: {X_centered.shape}, U: {U.shape}, S: {S.shape}, Vt: {Vt.shape}")
recon = U @ S @ Vt
print(f"Error: {np.mean(np.abs(X_centered - recon)):.2e}")

# 7. Reduce to 2D
k = 2
X_2d = X_centered @ Vt[:k, :].T
print(f"2D shape: {X_2d.shape}")

X: (36, 6), U: (36, 36), S: (36, 6), Vt: (6, 6)
Error: 5.35e-01
2D shape: (36, 2)


In [38]:
#SVD
import pandas as pd
import numpy as np

# 1. Load + Select numeric
train = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_test.csv')
X = train.iloc[:, 1:].values.astype(float)  # wine: 13 features

# 2. Center
X = X - X.mean(axis=0)

# 3. SVD (Simple & Correct)
def simple_svd(X):
    m, n = X.shape
    XtX = X.T @ X
    eigvals, V = np.linalg.eig(XtX)
    idx = np.argsort(eigvals)[::-1]
    V = V[:, idx]
    s = np.sqrt(np.maximum(eigvals[idx], 0))[:n]  # only n values
    U = X @ V[:, :n] / s
    return U, s, V[:, :n].T

U, s, Vt = simple_svd(X)

# 4. Reconstruct (CORRECT WAY)
S_diag = np.diag(s)           # <-- YEH ZAROORI HAI!
X_recon = U @ S_diag @ Vt     # <-- Ab shape match karega

# 5. Error
error = np.mean(np.abs(X - X_recon))

# 6. Print
print(f"Samples: {X.shape[0]}, Features: {X.shape[1]}")
print(f"Error: {error:.2e}")
print(f"SC1: {s[0]**2/sum(s**2)*100:.1f}%")
print(f"SC2: {s[1]**2/sum(s**2)*100:.1f}%")

Samples: 36, Features: 13
Error: 7.46e-14
SC1: 99.8%
SC2: 0.2%


In [45]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML_Datasets/wine_test.csv')
X_train = train.iloc[:,1:].values
y_train = np.where(train['target']==0, -1, 1)
X_test = test.iloc[:,1:].values
y_test = np.where(test['target']==0, -1, 1)

def stump_predict(X, f, t, d):
    p = np.ones(len(X))
    if d=='left': p[X[:,f]<t] = -1
    else: p[X[:,f]>=t] = -1
    return p

def adaboost(X, y, T=10):
    n = len(X); w = np.ones(n)/n; models = []
    for _ in range(T):
        best = (1,0,0,'left')
        for f in range(X.shape[1]):
            for t in np.unique(X[:,f]):
                for d in ['left','right']:
                    p = stump_predict(X,f,t,d)
                    e = np.sum(w[p!=y])
                    if e>0.5: e=1-e
                    if e<best[0]: best = (e,f,t,d)
        alpha = 0.5*np.log((1-best[0])/(best[0]+1e-10))
        p = stump_predict(X, best[1], best[2], best[3])
        w = w * np.exp(-alpha * y * p)
        w /= w.sum()
        models.append((alpha, best[1], best[2], best[3]))
    return models

# --- PREDICT (Direction use karo!) ---
def predict(X, models):
    s = np.zeros(len(X))
    for a, f, t, d in models:           # ← d yahan bhi chahiye
        s += a * stump(X, f, t, d)
    return np.sign(s)

# --- RUN ---
models = adaboost(X_train, y_train, T=15)
y_pred = predict(X_test, models)

# Agar reverse ho → fix
if np.mean(y_pred == y_test) < 0.5:
    y_pred = -y_pred
acc = np.mean(y_pred == y_test)
print(f" ACCURACY: {acc*100:.2f}%")

 ACCURACY: 88.89%


In [47]:
import pandas as pd
import numpy as np

# Load datasets
train = pd.read_csv('/content/drive/MyDrive/ML_Datasets/adult_train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML_Datasets/adult_test.csv')

# Select only numeric columns (like you did)
cols = [0, 2, 4, 10, 11, 12]
X_train = train.iloc[:, cols].values
X_test = test.iloc[:, cols].values

# Normalize features (mean=0, std=1, use train stats!)
def normalize(X, mean, std):
    return (X - mean) / (std + 1e-8)

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = normalize(X_train, mean, std)
X_test = normalize(X_test, mean, std)

# Convert target to -1 (for 0) and +1 (for 1)
y_train = np.where(train['target'] == 0, -1, 1)
y_test = np.where(test['target'] == 0, -1, 1)

# Decision stump function
def stump(X, feature, threshold, direction):
    pred = np.ones(X.shape[0])
    if direction == 'l':
        pred[X[:, feature] < threshold] = -1
    else:
        pred[X[:, feature] >= threshold] = -1
    return pred

# AdaBoost fit function
def adaboost(X, y, T=10):
    n = len(X)
    w = np.ones(n) / n
    models = []

    for _ in range(T):
        best_err = 1
        best_f, best_t, best_d = 0, 0, 'l'
        for f in range(X.shape[1]):
            thresholds = np.linspace(X[:, f].min(), X[:, f].max(), 20)
            for t in thresholds:
                for d in ['l', 'r']:
                    preds = stump(X, f, t, d)
                    err = np.sum(w[preds != y])
                    if err > 0.5:
                        err = 1 - err  # Use opposite
                    if err < best_err:
                        best_err = err
                        best_f, best_t, best_d = f, t, d
        # Check for degenerate error (avoid divide by zero)
        if best_err == 0:
            alpha = 10  # Large value
        else:
            alpha = 0.5 * np.log((1 - best_err) / (best_err + 1e-10))
        preds = stump(X, best_f, best_t, best_d)
        w = w * np.exp(-alpha * y * preds)
        w = w / w.sum()
        models.append((alpha, best_f, best_t, best_d))
    return models

# Prediction function
def predict(X, models):
    agg = np.zeros(X.shape[0])
    for alpha, f, t, d in models:
        agg += alpha * stump(X, f, t, d)
    return np.sign(agg)

# Train model
models = adaboost(X_train, y_train, T=15)

# Test
y_pred = predict(X_test, models)
acc = (y_pred == y_test).mean()
print(f" ACCURACY: {acc*100:.2f}%")


 ACCURACY: 82.17%
